<a href="https://colab.research.google.com/github/5y/202/blob/main/Reza_T2_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I try to understand the general preference mechanism behind a set of 1000 example selections. I'll try to guess the mechanism by looking at `t2_train.json` and check my guesses on `t2_test.json`.

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [2]:
!test -s "t2_train.json" || gdown "https://drive.google.com/uc?id=1zKeM0ld_51Q5lJGagiTv9lBWBHGfx7XL"
!test -s "t2_test.json" || gdown "https://drive.google.com/uc?id=1d1oeUOaZk8UHfoLyR5aOHTbtRMohFESn"

Downloading...
From: https://drive.google.com/uc?id=1zKeM0ld_51Q5lJGagiTv9lBWBHGfx7XL
To: /content/t2_train.json
100% 263k/263k [00:00<00:00, 58.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1d1oeUOaZk8UHfoLyR5aOHTbtRMohFESn
To: /content/t2_test.json
100% 261k/261k [00:00<00:00, 37.4MB/s]


In [ ]:
#@title Default title text
import json
import numpy as np
#call other libs
import pandas as pd
import pylab as pl
from sklearn import svm, linear_model
from sklearn.model_selection import train_test_split
import autokeras as ak

In [29]:
with open("t2_train.json") as f:
  train_data = json.load(f)

with open("t2_test.json") as f:
  test_data = json.load(f)



In [38]:

#using pandas to read json file from path
df_train = pd.read_json("t2_train.json")
df_test  = pd.read_json("t2_test.json")
#getting data about data
# df_train.info()
df_train

,options,selection
0,"[{'kind': 'c', 'attributes': [-2.27, 1.33, -0....",1
1,"[{'kind': 'c', 'attributes': [0.86, -0.74, 2.2...",1
2,"[{'kind': 'b', 'attributes': [-0.4, 0.16, -0.8...",2
3,"[{'kind': 'b', 'attributes': [0.38, -1.07, 1.0...",7
4,"[{'kind': 'a', 'attributes': [-0.66, -1.18, 0....",0
...,...,...
995,"[{'kind': 'c', 'attributes': [1.07, 1.01, -0.6...",1
996,"[{'kind': 'c', 'attributes': [-1.44, 1.44, 0.8...",4
997,"[{'kind': 'c', 'attributes': [-0.25, -1.15, 0....",5
998,"[{'kind': 'b', 'attributes': [2.12, 0.96]}, {'...",0


In [39]:
def convert(row):
  data = {}
  for i in range(1000):
    for n in range(3):
      if i < len(row) and n < len(row[i]['attributes']):
        data[f'attribute_{i}_{n}'] = row[i]['attributes'][n] 
      else:
          None
    if i<len(row):
      data[f'kind_{i}'] = row[i]['kind'] 
    else:
       None
  return pd.Series(data)

train_x = df_train['options'].apply(convert)
train_y = df_test['selection']


test_x = df_test['options'].apply(convert)
test_y = df_test ['selection']
df_train = pd.concat([train_x,train_y], axis=1).fillna(value=np.nan)
df_test = pd.concat([test_x,test_y], axis=1).fillna(value=np.nan)

df_train

,attribute_0_0,attribute_0_1,attribute_0_2,kind_0,attribute_1_0,attribute_1_1,attribute_1_2,kind_1,attribute_2_0,attribute_2_1,attribute_2_2,kind_2,attribute_3_0,attribute_3_1,attribute_3_2,kind_3,attribute_4_0,attribute_4_1,attribute_4_2,kind_4,attribute_5_0,attribute_5_1,attribute_5_2,kind_5,attribute_6_0,attribute_6_1,attribute_6_2,kind_6,attribute_7_0,attribute_7_1,kind_7,attribute_8_0,attribute_8_1,attribute_8_2,kind_8,attribute_9_0,attribute_9_1,attribute_9_2,kind_9,attribute_7_2,selection
0,-2.27,1.33,-0.84,c,1.97,1.27,-0.51,c,0.48,0.58,-0.18,c,1.41,-0.96,0.38,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.86,-0.74,2.27,c,-1.45,1.53,1.47,a,-0.89,-1.98,-0.35,c,0.16,-0.39,-0.30,c,-1.71,1.95,-0.51,b,-0.44,-1.61,-0.21,c,-0.90,0.39,-0.51,a,-1.18,-0.03,c,0.43,-0.63,-0.36,c,-0.81,-1.73,0.18,b,NaN,4
2,-0.40,0.16,-0.86,b,-0.84,0.38,-0.99,c,1.02,0.93,-0.90,a,1.84,1.53,-1.45,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,0.38,-1.07,1.05,b,0.21,0.98,0.36,c,0.71,0.13,0.40,c,-1.27,0.97,-1.17,b,1.94,1.92,1.48,a,-0.86,1.91,-0.27,c,0.80,0.61,0.92,b,0.38,-1.10,a,1.33,-0.44,NaN,b,1.85,-0.77,0.54,c,0.3,1
4,-0.66,-1.18,0.20,a,0.41,1.20,1.88,a,0.71,2.28,1.56,b,0.61,-0.58,-0.54,b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.07,1.01,-0.62,c,-1.33,0.88,-0.99,b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
996,-1.44,1.44,0.86,c,0.15,1.34,1.49,c,0.77,-1.03,0.60,c,0.95,0.87,0.60,c,-0.28,-1.25,0.71,b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
997,-0.25,-1.15,0.74,c,-2.30,0.53,0.00,c,-0.60,-0.99,1.51,c,0.74,2.39,-0.31,c,0.54,1.06,0.05,a,-0.11,-0.94,0.85,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
998,2.12,0.96,NaN,b,-0.53,1.71,1.19,b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [19]:

mymap = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5}
train= train_x.applymap(lambda s: mymap.get(s) if s in mymap else s)
test = test_x.applymap(lambda s: mymap.get(s) if s in mymap else s)



In [40]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=40
)
reg.fit(train_x, train_y, epochs=35)
# reg.predict(test_x,batch_size=32,verbose=1)
loss, acc =  reg.evaluate(test_x,y=test_y,batch_size=32,verbose=1)
print('Accuracy: %.3f' % acc)


Trial 40 Complete [00h 00m 05s]
val_loss: 3.7143657207489014

Best val_loss So Far: 3.599282741546631
Total elapsed time: 00h 05m 14s
INFO:tensorflow:Oracle triggered exit
Epoch 1/35
32/32 [==============================] - 1s 3ms/step - loss: 609.3417 - mean_squared_error: 609.3417
Epoch 2/35
32/32 [==============================] - 0s 3ms/step - loss: 7.8402 - mean_squared_error: 7.8402
Epoch 3/35
32/32 [==============================] - 0s 3ms/step - loss: 6.6248 - mean_squared_error: 6.6248
Epoch 4/35
32/32 [==============================] - 0s 3ms/step - loss: 5.5685 - mean_squared_error: 5.5685
Epoch 5/35
32/32 [==============================] - 0s 3ms/step - loss: 4.8337 - mean_squared_error: 4.8337
Epoch 6/35
32/32 [==============================] - 0s 3ms/step - loss: 4.3885 - mean_squared_error: 4.3885
Epoch 7/35
32/32 [==============================] - 0s 3ms/step - loss: 4.1459 - mean_squared_error: 4.1459
Epoch 8/35
32/32 [==============================] - 0s 3ms/step - lo

InvalidArgumentError: ignored

In [42]:
input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=40
)
clf.fit(train_x, train_y, epochs=30)
clf.predict(train_x)
model = clf.export_model()
model.summary()

accuarcy = clf.evaluate(test_x,test_y)
print(accuarcy[0],accuarcy[1])


Trial 40 Complete [00h 00m 11s]
val_loss: 1.8466520309448242

Best val_loss So Far: 1.7722246646881104
Total elapsed time: 00h 06m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
32/32 [==============================] - 1s 3ms/step - loss: 2.0817 - accuracy: 0.2530
Epoch 2/30
32/32 [==============================] - 0s 3ms/step - loss: 1.9042 - accuracy: 0.3080
Epoch 3/30
32/32 [==============================] - 0s 3ms/step - loss: 1.8679 - accuracy: 0.3050
Epoch 4/30
32/32 [==============================] - 0s 3ms/step - loss: 1.8413 - accuracy: 0.3240
Epoch 5/30
32/32 [==============================] - 0s 3ms/step - loss: 1.8243 - accuracy: 0.3070
Epoch 6/30
32/32 [==============================] - 0s 3ms/step - loss: 1.7747 - accuracy: 0.3330
Epoch 7/30
32/32 [==============================] - 0s 3ms/step - loss: 1.7647 - accuracy: 0.3280
Epoch 8/30
32/32 [==============================] - 0s 3ms/step - loss: 1.7345 - accuracy: 0.3460
Epoch 9/30
32/32 [=========================

InvalidArgumentError: ignored

I want to understand how well different scoring functions explain the choices seen in the training data. Regardless of how I define the scoring function, the evaluation code will always look the same. Suppose I think that the user always selects option with the *lowest* value of the second attribute. What fraction of the test data does this explain?

Trial 40 Complete [00h 00m 07s]
val_accuracy: 0.3214285671710968

Best val_accuracy So Far: 0.3511904776096344
Total elapsed time: 00h 08m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/35
32/32 [==============================] - 1s 3ms/step - loss: 2.1756 - accuracy: 0.2450
Epoch 2/35
32/32 [==============================] - 0s 3ms/step - loss: 1.9214 - accuracy: 0.2900
Epoch 3/35
32/32 [==============================] - 0s 3ms/step - loss: 1.8720 - accuracy: 0.2970
Epoch 4/35
32/32 [==============================] - 0s 3ms/step - loss: 1.8419 - accuracy: 0.3200
Epoch 5/35
32/32 [==============================] - 0s 3ms/step - loss: 1.8249 - accuracy: 0.3040
Epoch 6/35
32/32 [==============================] - 0s 3ms/step - loss: 1.8215 - accuracy: 0.2930
Epoch 7/35
32/32 [==============================] - 0s 3ms/step - loss: 1.7990 - accuracy: 0.2930
Epoch 8/35
32/32 [==============================] - 0s 3ms/step - loss: 1.8009 - accuracy: 0.3090
Epoch 9/35
32/32 [=================

InvalidArgumentError: ignored

In [ ]:
# np.random.seed(0)

# def random_baseline(option):
#   return np.random.random()
        
# def evaluate(data, scoring_fn):
#   correct = []
#   for example in data:
#     predicted = np.argmax([scoring_fn(o) for o in example['options']])
#     correct.append(predicted == example['selection'])
#   return dict(accuracy=np.mean(correct))

# print('Random:', evaluate(test_data, random_baseline))

# def my_terrible_score(option):
#   return -option['attributes'][1]

# print('Mine:', evaluate(train_data, my_terrible_score))

Random: {'accuracy': 0.271}
Mine: {'accuracy': 0.351}
